In [ ]:
import math
import numpy as np
# import tensorflow as tf
import cv2
import scipy.sparse as ssp
from scipy import ndimage
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
def interpolate_frames(frames):
    """
    与えられたフレーム列に対してフレーム補間を行い、新しいフレーム列を返す。

    :param frames: numpy.ndarray, 形状は (m, 128, 128)
    :return: numpy.ndarray, 形状は (2m-1, 128, 128)
    """
    m, h, w = frames.shape
    new_frames = []

    for i in range(m - 1):
        frame1 = frames[i].astype(np.float32)
        frame2 = frames[i + 1].astype(np.float32)

        # オプティカルフローの計算
        flow = cv2.calcOpticalFlowFarneback(
            prev=frame1,
            next=frame2,
            flow=None,
            pyr_scale=0.5,
            levels=3,
            winsize=15,
            iterations=3,
            poly_n=5,
            poly_sigma=1.2,
            flags=0
        )

        # 中間フレームの生成
        flow_half = flow * 0.5  # フローを半分にして中間時刻の流れを推定
        h_coords, w_coords = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
        remap_coords = np.stack((w_coords + flow_half[..., 0], h_coords + flow_half[..., 1]), axis=-1).astype(np.float32)

        interpolated_frame = cv2.remap(frame1, remap_coords, None, cv2.INTER_LINEAR)

        # フレームの格納
        # new_frames.append(frames[i])
        new_frames.append(interpolated_frame)

    # new_frames.append(frames[-1])  # 最後のフレームを追加

    return np.stack(new_frames)

In [ ]:
def matrix_to_tensor(H, m, n):
    return H.reshape(m, m, n, n)


def tensor_to_matrix(tensor, m, n):
    return tensor.reshape(m * m, n * n)


n = 128
_m = 128
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
# DATA_PATH = "../data"
EXP_DATE = "241105"
H_SETTING = "th_p-5_lmd-100_m-128"
H_mat = np.load(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_{H_SETTING}.npy")
print("H shape:", H_mat.shape, "type(H):", type(H_mat), "H.dtype:", H_mat.dtype)
H_ten = matrix_to_tensor(H_mat, _m, n)

In [ ]:
# フレーム補間
m = 2 * _m - 1  # 255
interpolated_tensor = np.zeros((m, m, n, n), dtype=np.float32)
for i in range(0, m, 2):
    frames = H_ten[i // 2, :, :, :]
    interpolated_frames = interpolate_frames(frames)
    for j in range(m):
        if j % 2 == 0:
            interpolated_tensor[i, j, :, :] = H_ten[i // 2, j // 2, :, :]
        else:
            interpolated_tensor[i, j, :, :] = interpolated_frames[j // 2, :, :]
for j in range(0, m):
    frames = interpolated_tensor[::2, j, :, :]
    interpolated_frames = interpolate_frames(frames)
    for i in range(1, m, 2):
        interpolated_tensor[i, j, :, :] = interpolated_frames[i // 2, :, :]

interpolated_matrix = tensor_to_matrix(interpolated_tensor, m, n)

In [ ]:
# np.save(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy", interpolated_matrix)
print("H shape:", interpolated_matrix.shape, "type(H):", "H.dtype:", interpolated_matrix.dtype)
# print(f"Saved {DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_int_{H_SETTING}.npy")

In [ ]:
import cupy as cp
SAMPLE_NAME = "Cameraman"
sample_image = Image.open(f"{DATA_PATH}/sample_image{n}/{SAMPLE_NAME}.png").convert("L")
sample_image = cp.asarray(sample_image).flatten() / 255

H_gpu = cp.asarray(interpolated_matrix)
Hf = H_gpu @ sample_image
Hf_img = cp.asnumpy(Hf.reshape(m, m))
Hf_img = np.clip(Hf_img, 0, 1)
print("Hf shape:", Hf_img.shape)
Hf_pil = Image.fromarray((Hf_img * 255).astype(np.uint8), mode="L")

fig, ax = plt.subplots(figsize=Hf_img.shape[::-1], dpi=1, tight_layout=True)
ax.imshow(Hf_pil, cmap="gray")
ax.axis("off")
plt.show()
FILENAME = f"{SAMPLE_NAME}_{H_SETTING}.png"
# fig.savefig(f"{DIRECTORY}/{FILENAME}", dpi=1)
# print(f"Saved {DIRECTORY}/{FILENAME}")

In [ ]:
J = 128
interpolated_frames = interpolated_tensor[:, J, :, :]
print("interpolated_frames.shape:", interpolated_frames.shape)
# 表示したいフレームの範囲を設定
start_index = 60
end_index = 84
frames_to_display = interpolated_frames[start_index:end_index + 1]

num_frames = len(frames_to_display)
cols = 5  # 列数を設定（必要に応じて変更可能）
rows = math.ceil(num_frames / cols)  # 行数を自動計算

# サブプロットを作成
fig, axs = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))

# サブプロットの軸を1次元にフラット化してループ処理
for idx, ax in enumerate(axs.flat):
    if idx < num_frames:
        frame = frames_to_display[idx]
        img = ax.imshow(frame, cmap='gray')
        plt.colorbar(img, ax=ax)
        ax.set_title(f'Frame {start_index + idx}')
    else:
        ax.axis('off')  # 使用しないサブプロットを非表示に

In [ ]:
import imageio
import io

def create_heatmap_gif(data, output_path, duration=0.5):
    """
    m×n×nのnumpy配列をヒートマップのGIFに変換する。

    Parameters:
    - data: ndarray, shape (m, n, n)
    - output_path: str, 保存するGIFのパス
    - duration: float, 各フレームの表示時間（秒）
    """
    # データが3次元か確認
    if data.ndim != 3:
        raise ValueError("入力データは3次元のnumpy配列である必要があります。")

    m, n1, n2 = data.shape
    if n1 != n2:
        raise ValueError("各スライスはn×nの形状である必要があります。")

    # テンソルの最大値と最小値を計算
    vmin = np.min(data)
    vmax = np.max(data)

    images = []  # GIFに追加する画像のリスト

    # 図と軸を設定
    fig, ax = plt.subplots(figsize=(6, 6))
    # plt.axis('off')  # 軸を非表示

    # ヒートマップを初期化
    cax = ax.imshow(data[0], cmap='gray', vmin=vmin, vmax=vmax)
    # カラーバーを一度だけ追加
    cbar = fig.colorbar(cax, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=8)
    title = ax.set_title(f'Frame 1/{m}', fontsize=10)

    for i in range(m):
        # ヒートマップのデータを更新
        cax.set_data(data[i])
        # タイトルを更新
        title.set_text(f'Frame {i+1}/{m}')

        # レイアウトを固定
        fig.tight_layout()

        # 画像をバッファに保存
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.v2.imread(buf)
        images.append(image)
        buf.close()

    plt.close(fig)  # 図を閉じる

    # GIFとして保存
    imageio.mimsave(output_path, images, duration=duration)
    print(f"GIFが保存されました: {output_path}")

create_heatmap_gif(interpolated_frames, f"{DATA_PATH}/{EXP_DATE}/interpolated_j{J}_video.gif", duration=0.2)